In [2]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

load_dotenv(override=True)
openai = OpenAI()

In [3]:
# Read the context from the pdf file
reader = PdfReader("me/mycv.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

 
  Page 1/3 
 
 
ANH DANG 
.NET Technical Lead 
Intermediate/Upper-Intermediate/Advanced in English 
Bachelor’s degree in Information Technology from Hanoi University 
of Science and Technology 
 
 
Highly accomplished Technical Leader and Certified Scrum Master with over 6 
years of experience, adept at driving successful project delivery within agile 
environments. Proven ability to cultivate strong client relationships, mentor high-
performing teams, and leverage robust problem-solving skills to navigate complex 
technical challenges. Expertise includes applying AI to build financial services and 
implementing Shift-Left methodologies for enhanced software development. 
 Led and coached cross-functional teams, ensuring successful project 
delivery through effective Scrum methodologies. 
 Partnered with clients to define requirements and align technical solutions 
with business objectives. 
 Applied strong analytical skills to diagnose and resolve complex technical 
issues, imple

In [4]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Anh Dang"


In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    print(history)
    return response.choices[0].message.content

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


[]


**Evaluator-optimizer**

In the evaluator-optimizer workflow, one LLM call generates a response while another provides evaluation and feedback in a loop.

In [5]:
# Define evaluator model. Use chat gpt 2.5 flash to evaluate response from chat gpt 4o mini
import os

gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")


In [27]:
# Handle generate

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

def generate(message, history):
    if "Anh Dang" in message:
        return "Ahh Anh Dang, You are a silly person."
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

def rerun_generate(message, history, reply, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


In [26]:
generate("Hi Anh Dang",[])

'Ahh Anh Dang, You are a silly person.'

In [29]:
# Handle evaluate

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

def evaluate(reply, message, history) -> Evaluation:
    evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
        You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
        The Agent is playing the role of {name} and is representing {name} on their website. \
        The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
        The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"
    evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
    evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

    evaluator_user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    evaluator_user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    evaluator_user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    evaluator_user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."

    # Use chat gpt 2.5 flash to evaluate response from chat gpt 4o mini
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

# evaluate(reply, "what's your name",[])

In [30]:
# Handle loop

def loop_chat(message, history):
    reply = generate(message, history)
    print(reply)
    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print(f"Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        feedback = evaluation.feedback
        print(feedback)
        reply = rerun_generate(message, history, reply, feedback)       
    return reply

In [ ]:
gr.ChatInterface(loop_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


My name is Anh Dang. How can I assist you today?
Passed evaluation - returning reply
Ahh Anh Dang, You are a silly person.
Failed evaluation - retrying
The Agent's response is unprofessional and inappropriate. Implying the user is silly is not aligned with the persona's instructions to be professional and engaging.
